In [ ]:
from doc2vec import read_fasta_to_kmers, encode_and_labels
import joblib
from model import train_doc2vec_model
from model_tools import learning_curve, evalution_metrics
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
# load data
train_pos = read_fasta_to_kmers('../data/pos_train_ds2.fasta')
train_neg = read_fasta_to_kmers('../data/neg_train_ds2.fasta')
test_pos = read_fasta_to_kmers('../data/pos_test_ds2.fasta')
test_neg = read_fasta_to_kmers('../data/neg_test_ds2.fasta')

In [ ]:
# encoding through Doc2Vec pretrained 
doc2vec_model = '../Doc2Vec_model/AFP_doc2vec_DS2.model'
train_data, train_labels = encode_and_labels(train_pos, train_neg, doc2vec_model)
test_data, test_labels = encode_and_labels(test_pos, test_neg, doc2vec_model)

## Random forest

In [ ]:
from sklearn import ensemble
forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(train_data, train_labels)
joblib.dump(forest, '../Doc2Vec_model/forest_doc2vec_DS2.pkl')

In [ ]:
forest = joblib.load('../Doc2Vec_model/forest_doc2vec_DS2.pkl')
labels_score = forest.predict(test_data)
evalution_metrics(test_labels, labels_score)

## Support vector machine

In [ ]:
from sklearn import svm
svc = svm.SVC()
svc_fit = svc.fit(train_data, train_labels)
joblib.dump(svc, '../Doc2Vec_model/svm_doc2vec_DS2.pkl')

In [ ]:
svc = joblib.load('../Doc2Vec_model/svm_doc2vec_DS2.pkl')
labels_score = svc.predict(test_data)
evalution_metrics(test_labels, labels_score)

## CNN

In [ ]:
train_data=train_data.reshape((train_data.shape[0],train_data.shape[1],1))
test_data=test_data.reshape((test_data.shape[0],test_data.shape[1],1))

In [ ]:
train_data_, test_data_, train_labels_, test_labels_ = train_test_split(train_data, train_labels, test_size= 0.1, random_state = 10, stratify = train_labels)


In [ ]:
t_m = train_doc2vec_model(train_data_, train_labels_, test_data_, test_labels_, 'doc2vec_DS2', path = '../Doc2Vec_model')

In [ ]:
learning_curve(t_m.history)

In [ ]:
model = load_model('../Doc2Vec_model/doc2vec_DS2_best_weights.h5')
labels_score = model.predict(test_data)
evalution_metrics(test_labels, labels_score)